In [ ]:
!pip install beautifulsoup4==4.12.3 finance-datareader==0.9.93 plotly==5.23.0 pykrx==1.0.48 yfinance==0.2.41 fredapi==0.5.2

In [2]:
from datetime import datetime, timedelta
import FinanceDataReader as fdr
from bs4 import BeautifulSoup
from fredapi import Fred
from pykrx import stock
import yfinance as yf
import pandas as pd
import requests
import os

file_path = './datas'
if not os.path.exists(file_path) :
    os.makedirs(file_path)



def get_per_pbr_df(ticker_symbol, start_date, end_date):
    # 주식 데이터 다운로드
    ticker = yf.Ticker(ticker_symbol)
    data = ticker.history(start=start_date, end=end_date)
    
    # 재무 데이터 다운로드
    financials = ticker.financials
    balance_sheet = ticker.balance_sheet
    
    # EPS 및 BVPS 계산
    try:
        net_income = financials.loc['Net Income'].iloc[0]  # 최신 데이터 사용
        shares_outstanding = ticker.info.get('sharesOutstanding', None)
        if shares_outstanding is None:
            raise ValueError("Shares outstanding not available")
        eps = net_income / shares_outstanding
        
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]  # 대체 가능한 키 사용
        book_value = total_assets - total_liabilities
        bvps = book_value / shares_outstanding
    except Exception as e:
        print(f"Error calculating EPS or BVPS: {e}")
        return pd.DataFrame()  # 빈 데이터프레임 반환
    
    # 기간 동안의 주가 데이터를 기반으로 PER 및 PBR 계산
    per_list = []
    pbr_list = []
    dates = []
    
    for date, row in data.iterrows():
        avg_price = row['Close']
        per = avg_price / eps
        pbr = avg_price / bvps
        per_list.append(per)
        pbr_list.append(pbr)
        dates.append(date)
    
    # 데이터프레임 생성
    result_df = pd.DataFrame({
        'Date': dates,
        'PER': per_list,
        'PBR': pbr_list
    })
    
    result_df.set_index('Date', inplace=True)
    return result_df

def check_time_data(check_time) :
    if 'Q1' in check_time :
        check_time = f'{check_time[:4]}0101'
    elif 'Q2' in check_time :
        check_time = f'{check_time[:4]}0401'
    elif 'Q3' in check_time :
        check_time = f'{check_time[:4]}0701'
    elif 'Q4' in check_time :
        check_time = f'{check_time[:4]}1001'
    elif len(check_time) <= 4 :
        check_time = f'{check_time}0101'
    elif len(check_time) <= 6 :
        check_time = f'{check_time}01'
    return check_time

def convert_date_format(date_str):
    return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"


print('============= Program Start =============')
print('Start time : ', datetime.today())
start_day = '2014-01-01'
month_date = str(datetime.now().month)
day_date = str(datetime.now().day)
if len(month_date) == 1 :
    month_date = f'0{month_date}'
if len(day_date) == 1 :
    day_date = f'0{day_date}'
end_day = f'{datetime.now().year}-{month_date}-{day_date}' #'2024-08-28'

start_date = datetime(int(start_day[:4]), int(start_day[5:7]), int(start_day[8:10]))
end_date = datetime(int(end_day[:4]), int(end_day[5:7]), int(end_day[8:10]))
#end_date = f'{datetime.now().year}{datetime.now().month}{datetime.now().day}'

date_df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date))

df_len = []

# ************************************** tb_stock **************************************
fdr_start_year = start_day[:4]

# Samsung(005930)
samsung_stock = fdr.DataReader('005930', fdr_start_year) 
samsung_stock.rename(columns={'Close' : 'sc_ss_stock'}, inplace=True)
samsung_stock = samsung_stock['sc_ss_stock']

# Samsung PER, PBR, ROE
samsung_PER_PBR_ROE = stock.get_market_fundamental(start_day, end_day, "005930")[['PER', 'PBR']] # 삼성전자
samsung_PER_PBR_ROE.rename(columns={'PER' : 'sc_ss_per', 'PBR' : 'sc_ss_pbr'}, inplace=True)
samsung_PER_PBR_ROE['sc_ss_roe'] = samsung_PER_PBR_ROE['sc_ss_pbr'] / samsung_PER_PBR_ROE['sc_ss_per']

# Samsung Market Capitalization
ticker = "005930.KS"
samsung_data = yf.download(ticker, start=start_date, end=end_date)
close_prices = samsung_data['Close']
samsung = yf.Ticker(ticker)
shares_outstanding = samsung.info['sharesOutstanding']
samsung_market = close_prices * shares_outstanding
samsung_Market_Cap = pd.DataFrame({'sc_ss_mc': samsung_market})
samsung_Market_Cap['sc_ss_mc'] = samsung_Market_Cap['sc_ss_mc'].astype(str).map(lambda x : x+'00000')

# Apple(AAPL)
# Apple Stock
apple_stock = fdr.DataReader('AAPL', fdr_start_year) 
apple_stock.rename(columns={'Close' : 'sc_ap_stock'}, inplace=True)
apple_stock = apple_stock['sc_ap_stock']

# Apple PER, PBR, ROE
apple_PER_PBR_ROE = get_per_pbr_df("AAPL", start_day, end_day)
apple_PER_PBR_ROE.rename(columns={'PER' : 'sc_ap_per', 'PBR' : 'sc_ap_pbr'}, inplace=True)
apple_PER_PBR_ROE = apple_PER_PBR_ROE.reset_index(drop=False)
apple_PER_PBR_ROE['Date'] = apple_PER_PBR_ROE['Date'].astype(str).map(lambda x : x[:10])
apple_PER_PBR_ROE['Date'] = pd.to_datetime(apple_PER_PBR_ROE['Date'])
apple_PER_PBR_ROE.set_index(keys='Date', inplace=True)
apple_PER_PBR_ROE['sc_ap_roe'] = apple_PER_PBR_ROE['sc_ap_pbr'].astype(float) / apple_PER_PBR_ROE['sc_ap_per'].astype(float)

# Apple Market Capitalization
ticker = 'AAPL'
apple = yf.Ticker(ticker)
apple_data = apple.history(start=start_date, end=end_date)
apple_Market_Cap = apple.info['sharesOutstanding'] * apple_data['Close']
apple_Market_Cap = pd.DataFrame({'sc_ap_mc': apple_Market_Cap})
apple_Market_Cap = apple_Market_Cap.reset_index()
apple_Market_Cap['Date'] = apple_Market_Cap['Date'].astype(str).map(lambda x : x[:10])
apple_Market_Cap['Date'] = pd.to_datetime(apple_Market_Cap['Date'])
apple_Market_Cap.set_index(keys='Date', inplace=True)
apple_Market_Cap['sc_ap_mc'] = apple_Market_Cap['sc_ap_mc'].astype(str).map(lambda x : x+'000')

# Bit-Coin(BTC)
bitcoin_stock = fdr.DataReader('BTC/USD', fdr_start_year) 
bitcoin_stock.rename(columns={'Close' : 'sc_coin'}, inplace=True)
bitcoin_stock = bitcoin_stock['sc_coin']

# stock table insert
tb_stock_df_list = [samsung_stock, samsung_PER_PBR_ROE, samsung_Market_Cap,
                    apple_stock, apple_PER_PBR_ROE, apple_Market_Cap, 
                    bitcoin_stock]

stock_df = date_df.copy()
for i in range(len(tb_stock_df_list)) :
    stock_df = stock_df.join(tb_stock_df_list[i])

stock_df = stock_df.reset_index().rename(columns={'index' : 'fd_date'})
stock_df.fillna(method='ffill', inplace=True)
stock_df['fd_date'] = stock_df['fd_date'].astype(str).map(lambda x : x[:10])







# ************************************** tb_main_economic_index **************************************
# NASDAQ
nasdaq = fdr.DataReader('IXIC', fdr_start_year)
nasdaq.rename(columns = {'Close' : 'mei_nasdaq'}, inplace=True)
nasdaq = nasdaq['mei_nasdaq']

# S&P500
snp500 = fdr.DataReader('S&P500', fdr_start_year)
snp500.rename(columns = {'Close' : 'mei_sp500'}, inplace=True)
snp500 = snp500['mei_sp500']

# Dow
Dow = fdr.DataReader('DJI', fdr_start_year)
Dow.rename(columns = {'Close' : 'mei_dow'}, inplace=True)
Dow = Dow['mei_dow']

# KOSPI
kospi = fdr.DataReader('KS11', fdr_start_year)
kospi.rename(columns = {'Close' : 'mei_kospi'}, inplace=True)
kospi = kospi['mei_kospi']

# Gold, Oil
today_date = datetime.today()-timedelta(1)
days_passed = (today_date - start_date).days

# Gold
gold = yf.Ticker('GC=F')
gold_data = gold.history(period='max').tail(days_passed)
gold_data.rename(columns={'Close' : 'mei_gold'}, inplace=True)
gold_data = gold_data['mei_gold']
gold_data = gold_data.reset_index(drop=False)
gold_data['Date'] = gold_data['Date'].astype(str).map(lambda x : x[:10])
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
gold_data.set_index(keys='Date', inplace=True)

# Oil
oil = yf.Ticker('BZ=F')
oil_data = oil.history(period='max').tail(days_passed)
oil_data.rename(columns={'Close' : 'mei_oil'}, inplace=True)
oil_data = oil_data['mei_oil']
oil_data = oil_data.reset_index(drop=False)
oil_data['Date'] = oil_data['Date'].astype(str).map(lambda x : x[:10])
oil_data['Date'] = pd.to_datetime(oil_data['Date'])
oil_data.set_index(keys='Date', inplace=True)

# Exchange Rate
dollar_to_won = fdr.DataReader('USD/KRW', fdr_start_year)
dollar_to_won.rename(columns={'Close' : 'mei_ex_rate'}, inplace=True)
dollar_to_won = dollar_to_won['mei_ex_rate']

# main economic index table insert
tb_main_economic_index_df_list = [nasdaq, snp500, Dow, kospi, gold_data, oil_data, dollar_to_won]

main_economic_index_df = date_df.copy()
for i in range(len(tb_main_economic_index_df_list)) :
    main_economic_index_df = main_economic_index_df.join(tb_main_economic_index_df_list[i])

main_economic_index_df = main_economic_index_df.reset_index().rename(columns={'index' : 'fd_date'})
main_economic_index_df.fillna(method='ffill', inplace=True)
main_economic_index_df['fd_date'] = main_economic_index_df['fd_date'].astype(str).map(lambda x : x[:10])






# ************************************** tb_korea_economic_indicator **************************************
data_name_ko = ['국내 총 생산',
            'M2 통화공급 (말잔)',
            'M2 통화공급 (평잔)',
            '중앙은행 기준금리',
            '생산자물가지수',
            '수입물가지수',
            '소비자물가지수',
            '수입',
            '수출',
            '소비자심리지수',
            '기업경기실사지수']

data_name = ['kei_gdp',
            'kei_m2_end',
            'kei_m2_avg',
            'kei_fed_rate',
            'kei_ppi',
            'kei_ipi',
            'kei_cpi',
            'kei_imp',
            'kei_exp',
            'kei_cs',
            'kei_bsi']

my_key = '5MYJ8MSDU0644HI8BXXH'

api_link = ['/200Y102/Q/2014Q1/2024Q2/10111',
            '/101Y007/M/201401/202406/BBIA00',
            '/101Y008/M/201401/202406/BBJA00',
            '/722Y001/M/201401/202406/0101000',
            '/404Y014/M/201401/202406/*AA',
            '/401Y015/M/201401/202406/*AA/W',
            '/901Y009/M/201401/202406/0',
            '/403Y003/M/201401/202406/*AA',
            '/403Y001/M/201401/202406/*AA',
            '/511Y002/M/201401/202406/FME/99988',
            '/512Y007/M/201401/202406/AA/99988']

all_data = []
all_time = []
for i in range(len(api_link)) :
    value_time = []
    value_data = []
    search_url = f'https://ecos.bok.or.kr/api/StatisticSearch/{my_key}/xml/kr/1/1{api_link[i]}'

    search_respons = requests.get(search_url)
    search_xml = search_respons.text
    search_soup = BeautifulSoup(search_xml, 'xml')
    total_val = search_soup.find('list_total_count')

    url = f'https://ecos.bok.or.kr/api/StatisticSearch/{my_key}/xml/kr/1/{total_val.text}{api_link[i]}'
    respons = requests.get(url)
    title_xml = respons.text
    title_soup = BeautifulSoup(title_xml, 'xml') 
    value_d = title_soup.find_all('DATA_VALUE')
    value_t = title_soup.find_all('TIME')
    for j in value_d : 
        value_data.append(j.text)
    for j in value_t :
        check_time = check_time_data(j.text)
        value_time.append(check_time)
    all_time.append(value_time)
    all_data.append(value_data)

all_time = [[convert_date_format(date) for date in row] for row in all_time]

ca = pd.read_excel('./kei_ca.xlsx')
ca['Date'] = pd.to_datetime(ca['Date'])
ca = ca.set_index(keys='Date')

fers = pd.read_excel('./fer_data.xlsx').transpose().iloc[1:].reset_index(drop=False).rename(columns={'index' : 'Date'})
fers['Date'] = fers['Date'].astype(str).map(lambda x : x[:4]+'-'+x[4:6]+'-01')
fers['Date'] = pd.to_datetime(fers['Date'])
fers = fers.set_index(keys='Date')
fers.columns = ['kei_fr']
fers['kei_fr'] = fers['kei_fr'].str.replace(',', '').astype(int)

# korea economic indicator table insert

korea_economic_indicator_df = date_df.copy()
for i in range(0, 11) :
    ko_eco_indi = pd.DataFrame({'Date' :  pd.to_datetime(all_time[i]), data_name[i] : all_data[i]}).set_index('Date')
    korea_economic_indicator_df = korea_economic_indicator_df.join(ko_eco_indi)
korea_economic_indicator_df = korea_economic_indicator_df.join(fers).join(ca).reset_index().rename(columns={'index' : 'fd_date'})

korea_economic_indicator_df.fillna(method='ffill', inplace=True)
korea_economic_indicator_df['fd_date'] = korea_economic_indicator_df['fd_date'].astype(str).map(lambda x : x[:10])







# ************************************** tb_us_economic_indicator **************************************
fred_api_key = '8c2950ebb8aec46ba6e5f2765e00582e'
fred = Fred(api_key=fred_api_key)

indicators = {
    "uei_gdp": "GDP",
    "uei_fed_rate": "FEDFUNDS",
    "uei_ipi": "IR",
    "uei_ppi": "PPIACO",
    "uei_cpi": "CPIAUCSL",
    "uei_cpi_m": "CPIAUCNS",
    "uei_trade": "BOPGSTB",
    "uei_cb_cc": "CSCICP03USM665S",
    "uei_ps_m": "PCE",
    "uei_rs_m": "RSXFS",
    "uei_umich_cs": "UMCSENT"
}

us_economic_indicator_dic = {}
for name, series_id in indicators.items():
    try:
        us_economic_indicator_dic[name] = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
    except ValueError as e:
        print(f"Error fetching {name}: {e}")

us_economic_indicator_df = date_df.copy()
wei_dic = pd.DataFrame(us_economic_indicator_dic)
us_economic_indicator_df = us_economic_indicator_df.join(wei_dic)
us_economic_indicator_df = us_economic_indicator_df.reset_index().rename(columns={'index' : 'fd_date'})
us_economic_indicator_df.fillna(method='ffill', inplace=True)
us_economic_indicator_df['fd_date'] = us_economic_indicator_df['fd_date'].astype(str).map(lambda x : x[:10])




# tb_finance_date
date_df = date_df.reset_index().rename(columns={'index' : 'fd_date'})
date_df['fd_date'] = date_df['fd_date'].astype(str).map(lambda x : x[:10])

# Excel save
df_len.append(len(stock_df.dropna(axis=0)))
df_len.append(len(main_economic_index_df.dropna(axis=0)))
df_len.append(len(korea_economic_indicator_df.dropna(axis=0)))
df_len.append(len(us_economic_indicator_df.dropna(axis=0)))
df_len = min(df_len)

date_df.tail(df_len).to_excel(f'{file_path}/tb_finance_date.xlsx', index=False)
stock_df.tail(df_len).to_excel(f'{file_path}/tb_stock.xlsx', index=False)
main_economic_index_df.tail(df_len).to_excel(f'{file_path}/tb_main_economic_index.xlsx', index=False)
korea_economic_indicator_df.tail(df_len).to_excel(f'{file_path}/tb_korea_economic_indicator.xlsx', index=False)
us_economic_indicator_df.tail(df_len).to_excel(f'{file_path}/tb_us_economic_indicator.xlsx', index=False)

print('============= Excel saved complete =============')

============= Program Start =============
Start time :  2024-09-04 14:50:31.223486


[*********************100%%**********************]  1 of 1 completed
C:\Users\reals\AppData\Local\Temp\ipykernel_13864\3710058901.py:168: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_df.fillna(method='ffill', inplace=True)


1 001 2014-01-01 00:00:00 2015-12-31 00:00:00
1 001 2016-01-01 00:00:00 2017-12-31 00:00:00
1 001 2018-01-01 00:00:00 2019-12-31 00:00:00
1 001 2020-01-01 00:00:00 2021-12-31 00:00:00
1 001 2022-01-01 00:00:00 2023-12-31 00:00:00
1 001 2024-01-01 00:00:00 2025-12-31 00:00:00


C:\Users\reals\AppData\Local\Temp\ipykernel_13864\3710058901.py:235: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_economic_index_df.fillna(method='ffill', inplace=True)
C:\Users\reals\AppData\Local\Temp\ipykernel_13864\3710058901.py:329: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  korea_economic_indicator_df.fillna(method='ffill', inplace=True)
C:\Users\reals\AppData\Local\Temp\ipykernel_13864\3710058901.py:367: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  us_economic_indicator_df.fillna(method='ffill', inplace=True)
C:\Users\reals\AppData\Local\Temp\ipykernel_13864\3710058901.py:384: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  date_df.tail

============= Excel saved complete =============


In [22]:
a = f'{datetime.now().year}-{datetime.now().month}-{datetime.now().day}'
a

'2024-8-14'

In [9]:
date_df.reset_index(drop=False)

,index
0,2014-01-01
1,2014-01-02
2,2014-01-03
3,2014-01-04
4,2014-01-05
...,...
3894,2024-08-30
3895,2024-08-31
3896,2024-09-01
3897,2024-09-02


In [8]:
date_df.index

DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04',
               '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',
               '2014-01-09', '2014-01-10',
               ...
               '2024-08-25', '2024-08-26', '2024-08-27', '2024-08-28',
               '2024-08-29', '2024-08-30', '2024-08-31', '2024-09-01',
               '2024-09-02', '2024-09-03'],
              dtype='datetime64[ns]', length=3899, freq='D')